In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

In [12]:
df = pd.read_csv('data/Covid Live.csv', encoding_errors='ignore').drop('#', axis=1)
df

,"Country,\nOther",Total\nCases,Total\nDeaths,New\nDeaths,Total\nRecovered,Active\nCases,"Serious,\nCritical",Tot Cases/\n1M pop,Deaths/\n1M pop,Total\nTests,Tests/\n1M pop,Population
0,USA,"98,166,904","1,084,282",NaN,"94,962,112","2,120,510","2,970","293,206","3,239","1,118,158,870","3,339,729","334,805,269"
1,India,"44,587,307","528,629",NaN,"44,019,095","39,583",698,"31,698",376,"894,416,853","635,857","1,406,631,776"
2,France,"35,342,950","155,078",NaN,"34,527,115","660,757",869,"538,892","2,365","271,490,188","4,139,547","65,584,518"
3,Brazil,"34,706,757","686,027",NaN,"33,838,636","182,094","8,318","161,162","3,186","63,776,166","296,146","215,353,593"
4,Germany,"33,312,373","149,948",NaN,"32,315,200","847,225","1,406","397,126","1,788","122,332,384","1,458,359","83,883,596"
...,...,...,...,...,...,...,...,...,...,...,...,...
225,Niue,80,NaN,NaN,80,0,NaN,"49,322",NaN,NaN,NaN,"1,622"
226,Vatican City,29,NaN,NaN,29,0,NaN,"36,295",NaN,NaN,NaN,799
227,Tuvalu,20,NaN,NaN,NaN,20,NaN,"1,658",NaN,NaN,NaN,"12,066"
228,Western Sahara,10,1,NaN,9,0,NaN,16,2,NaN,NaN,"626,161"


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Country,
Other     230 non-null    object 
 1   Total
Cases        230 non-null    object 
 2   Total
Deaths       225 non-null    object 
 3   New
Deaths         3 non-null      float64
 4   Total
Recovered    214 non-null    object 
 5   Active
Cases       215 non-null    object 
 6   Serious,
Critical  147 non-null    object 
 7   Tot Cases/
1M pop  228 non-null    object 
 8   Deaths/
1M pop     223 non-null    object 
 9   Total
Tests        214 non-null    object 
 10  Tests/
1M pop      214 non-null    object 
 11  Population         228 non-null    object 
dtypes: float64(1), object(11)
memory usage: 21.7+ KB
